In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import json
from sklearn.utils import shuffle
pd.set_option('display.max_columns', None)
from sklearn.linear_model import Ridge, Lasso, LinearRegression
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score

with open('/content/new_pet_30_columns (1).json', 'r', encoding='utf-8') as f:
    df = json.load(f)
df = pd.DataFrame.from_dict(df).T

df = shuffle(df).reset_index(drop=True)

# Cái này bỏ bớt outlier vì khả năng cao sẽ ảnh hưởng tới độ chính xác của mô hình
df = df[df['price'] < 10_000_000]
# Chia price nhỏ nhỏ để ra số chấp nhận được, số lớn quá sẽ thành NaN
df['price'] = df['price'] / 1_000_000
# Chọn ra một số thuộc tính hay ho
# df = df[['pet_type_name', 'pet_breed_name', 'pet_age_name', 'pet_size', 'area_name', 'price']].reset_index(drop=True)

df

FileNotFoundError: [Errno 2] No such file or directory: '/content/new_pet_30_columns (1).json'

# 2.1 'pet_type_name, ''pet_breed_name', 'pet_age_name', 'pet_size_name', 'longitude','latitude'

In [ ]:
df_2_1 = df[['pet_type_name', 'pet_breed_name', 'pet_age_name', 'pet_size_name', 'longitude','latitude','price', 'area_name', 'ward_name', 'region_name']]
df_2_1.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5148 entries, 0 to 5307
Data columns (total 10 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   pet_type_name   5148 non-null   object
 1   pet_breed_name  5148 non-null   object
 2   pet_age_name    5148 non-null   object
 3   pet_size_name   5148 non-null   object
 4   longitude       5148 non-null   object
 5   latitude        5148 non-null   object
 6   price           5148 non-null   object
 7   area_name       5148 non-null   object
 8   ward_name       5148 non-null   object
 9   region_name     5148 non-null   object
dtypes: object(10)
memory usage: 442.4+ KB


In [ ]:
from sklearn.preprocessing import LabelEncoder
breed, age, size, type, area, ward, region = LabelEncoder(), LabelEncoder(), LabelEncoder(), LabelEncoder(), LabelEncoder(), LabelEncoder(), LabelEncoder()

df_2_1['breed'] = breed.fit_transform(df_2_1['pet_breed_name'])
df_2_1['age'] = age.fit_transform(df_2_1['pet_age_name'])
df_2_1['size'] = size.fit_transform(df_2_1['pet_size_name'])
df_2_1['type'] = type.fit_transform(df_2_1['pet_type_name'])
df_2_1['area'] = area.fit_transform(df_2_1['area_name'])
df_2_1['ward'] = ward.fit_transform(df_2_1['ward_name'])
df_2_1['region'] = region.fit_transform(df_2_1['region_name'])
df_2_1

<ipython-input-24-2c911a12f7c4>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_2_1['breed'] = breed.fit_transform(df_2_1['pet_breed_name'])
<ipython-input-24-2c911a12f7c4>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_2_1['age'] = age.fit_transform(df_2_1['pet_age_name'])
<ipython-input-24-2c911a12f7c4>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https:

,pet_type_name,pet_breed_name,pet_age_name,pet_size_name,longitude,latitude,price,area_name,ward_name,region_name,breed,age,size,type,area,ward,region
0,Chó,Chó Phốc Sóc,Chó con (dưới 3 tháng tuổi),Siêu nhỏ (tiny),106.685135,10.950786,5.5,Thành phố Thuận An,Phường An Thạnh,Bình Dương,49,2,3,1,284,46,2
1,Chó,Chó Pug,Chó nhỏ (dưới 1 năm tuổi),Nhỏ,105.69111,21.013826,2.268686,Huyện Hoài Đức,Xã Song Phương,Hà Nội,52,3,2,1,58,895,16
2,Chó,Chó Golden,Chó trưởng thành (hơn 1 tuổi),Lớn,106.59671,10.868425,8.3,Huyện Hóc Môn,Xã Tân Xuân,Tp Hồ Chí Minh,40,4,1,1,66,1005,44
3,Chó,Chó Samoyed,Chó nhỏ (dưới 1 năm tuổi),Nhỏ,107.23118,10.705099,5.5,Huyện Châu Đức,Xã Kim Long,Bà Rịa - Vũng Tàu,54,3,2,1,22,732,1
4,Chó,Chó Becgie,Chó nhỏ (dưới 1 năm tuổi),Lớn,105.94061,21.22481,0.16,Huyện Yên Phong,Xã Tam Giang,Bắc Ninh,29,3,1,1,181,907,9
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5303,Chim,Chim Bồ Câu,Chim lớn (từ 3 tháng tuổi),Khác (không xác định được),106.58481,10.716207,0.22,Huyện Bình Chánh,Xã Tân Kiên,Tp Hồ Chí Minh,1,0,0,0,7,980,44
5304,Chó,Chó Bull,Chó con (dưới 3 tháng tuổi),Nhỏ,106.48673,10.9941,5.0,Huyện Củ Chi,Xã Tân An Hội,Tp Hồ Chí Minh,30,2,2,1,42,969,44
5305,Mèo,Mèo Sphynx,Mèo con (dưới 3 tháng tuổi),Khác (không xác định được),106.62026,10.87035,6.5,Quận 12,Phường Tân Chánh Hiệp,Tp Hồ Chí Minh,79,8,0,3,204,388,44
5306,Chim,Chim Họa Mi,Chim lớn (từ 3 tháng tuổi),Khác (không xác định được),106.77654,10.725062,0.9,Huyện Nhơn Trạch,Xã Phú Hữu,Đồng Nai,8,0,0,0,103,843,52


In [ ]:
X = df_2_1.drop(['price', 'pet_breed_name', 'pet_age_name', 'pet_size_name', 'price', 'pet_type_name', 'area_name', 'ward_name', 'region_name'], axis = 1).values
y = df_2_1.price.values

X[0]

array([106.685135, 10.950786, 49, 2, 3, 1, 284, 46, 2], dtype=object)

In [ ]:
X.shape

(5148, 9)

In [ ]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
ct = ColumnTransformer([('onehot', OneHotEncoder(), [2, 3, 4, 5, 6, 7, 8])], remainder = 'passthrough')

X = ct.fit_transform(X)
X.toarray()[0]

array([  0.      ,   0.      ,   0.      , ...,   0.      , 106.685135,
        10.950786])

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state=42)

model_1 = LinearRegression()
model_1.fit(X_train, y_train)
a = model_1.score(X_test, y_test)
a

0.4168278769152166

# Cách 2.2 : 'pet_type','pet_breed', 'pet_age', 'pet_size', 'longitude','latitude'

In [ ]:
df_2_2 = df[['pet_type','pet_breed', 'pet_age', 'pet_size', 'longitude','latitude','price', 'area_name', 'ward_name', 'region_name']]
df_2_2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5148 entries, 0 to 5307
Data columns (total 10 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   pet_type     5148 non-null   object
 1   pet_breed    5148 non-null   object
 2   pet_age      5148 non-null   object
 3   pet_size     5148 non-null   object
 4   longitude    5148 non-null   object
 5   latitude     5148 non-null   object
 6   price        5148 non-null   object
 7   area_name    5148 non-null   object
 8   ward_name    5148 non-null   object
 9   region_name  5148 non-null   object
dtypes: object(10)
memory usage: 442.4+ KB


In [ ]:
breed, age, size, type, area, ward, region = LabelEncoder(), LabelEncoder(), LabelEncoder(), LabelEncoder(), LabelEncoder(), LabelEncoder(), LabelEncoder()

df_2_2['type'] = type.fit_transform(df_2_2['pet_type'])
df_2_2['breed'] = breed.fit_transform(df_2_2['pet_breed'])
df_2_2['age'] = age.fit_transform(df_2_2['pet_age'])
df_2_2['size'] = size.fit_transform(df_2_2['pet_size'])
df_2_2['area'] = area.fit_transform(df_2_2['area_name'])
df_2_2['ward'] = ward.fit_transform(df_2_2['ward_name'])
df_2_2['region'] = region.fit_transform(df_2_2['region_name'])
df_2_2

<ipython-input-30-dd0becb76f75>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_2_2['type'] = type.fit_transform(df_2_2['pet_type'])
<ipython-input-30-dd0becb76f75>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_2_2['breed'] = breed.fit_transform(df_2_2['pet_breed'])
<ipython-input-30-dd0becb76f75>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://panda

,pet_type,pet_breed,pet_age,pet_size,longitude,latitude,price,area_name,ward_name,region_name,type,breed,age,size,area,ward,region
0,12020.0,17.0,1.0,1.0,106.685135,10.950786,5.5,Thành phố Thuận An,Phường An Thạnh,Bình Dương,1,16,0,0,284,46,2
1,12020.0,21.0,2.0,2.0,105.69111,21.013826,2.268686,Huyện Hoài Đức,Xã Song Phương,Hà Nội,1,20,1,1,58,895,16
2,12020.0,11.0,3.0,4.0,106.59671,10.868425,8.3,Huyện Hóc Môn,Xã Tân Xuân,Tp Hồ Chí Minh,1,10,2,3,66,1005,44
3,12020.0,23.0,2.0,2.0,107.23118,10.705099,5.5,Huyện Châu Đức,Xã Kim Long,Bà Rịa - Vũng Tàu,1,22,1,1,22,732,1
4,12020.0,3.0,2.0,4.0,105.94061,21.22481,0.16,Huyện Yên Phong,Xã Tam Giang,Bắc Ninh,1,2,1,3,181,907,9
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5303,12030.0,1.0,2.0,5.0,106.58481,10.716207,0.22,Huyện Bình Chánh,Xã Tân Kiên,Tp Hồ Chí Minh,2,0,1,4,7,980,44
5304,12020.0,4.0,1.0,2.0,106.48673,10.9941,5.0,Huyện Củ Chi,Xã Tân An Hội,Tp Hồ Chí Minh,1,3,0,1,42,969,44
5305,12050.0,8.0,1.0,5.0,106.62026,10.87035,6.5,Quận 12,Phường Tân Chánh Hiệp,Tp Hồ Chí Minh,3,7,0,4,204,388,44
5306,12030.0,5.0,2.0,5.0,106.77654,10.725062,0.9,Huyện Nhơn Trạch,Xã Phú Hữu,Đồng Nai,2,4,1,4,103,843,52


In [ ]:
X = df_2_2.drop(['price', 'pet_type','pet_breed', 'pet_age', 'pet_size', 'area_name', 'ward_name', 'region_name'], axis = 1).values
y = df_2_2.price.values

X, y

(array([[106.685135, 10.950786, 1, ..., 284, 46, 2],
        [105.69111, 21.013826, 1, ..., 58, 895, 16],
        [106.59671, 10.868425, 1, ..., 66, 1005, 44],
        ...,
        [106.62026, 10.87035, 3, ..., 204, 388, 44],
        [106.77654, 10.725062, 2, ..., 103, 843, 52],
        [105.81582, 21.02089, 3, ..., 211, 334, 16]], dtype=object),
 array([5.5, 2.268686, 8.3, ..., 6.5, 0.9, 0.15], dtype=object))

In [ ]:
ct = ColumnTransformer([('onehot', OneHotEncoder(), [2, 3, 4, 5, 6, 7, 8])], remainder = 'passthrough')

X = ct.fit_transform(X)
X.toarray()[0]

array([  0.      ,   1.      ,   0.      , ...,   0.      , 106.685135,
        10.950786])

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state=42)

model_2 = LinearRegression()
model_2.fit(X_train, y_train)
e = model_2.score(X_test, y_test)
e

0.2261336868368391

# 2.3 Them pet_breed

In [2]:
df_2_3 = df[['pet_breed', 'pet_breed_name', 'pet_age_name', 'pet_size_name', 'pet_type_name', 'longitude','latitude','price', 'area_name', 'ward_name', 'region_name']]
df_2_3.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5148 entries, 0 to 5307
Data columns (total 11 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   pet_breed       5148 non-null   object
 1   pet_breed_name  5148 non-null   object
 2   pet_age_name    5148 non-null   object
 3   pet_size_name   5148 non-null   object
 4   pet_type_name   5148 non-null   object
 5   longitude       5148 non-null   object
 6   latitude        5148 non-null   object
 7   price           5148 non-null   object
 8   area_name       5148 non-null   object
 9   ward_name       5148 non-null   object
 10  region_name     5148 non-null   object
dtypes: object(11)
memory usage: 482.6+ KB


In [ ]:
breed, age, size, type, b, area, ward, region = LabelEncoder(), LabelEncoder(), LabelEncoder(), LabelEncoder(), LabelEncoder(), LabelEncoder(), LabelEncoder(), LabelEncoder()

df_2_3['type'] = type.fit_transform(df_2_3['pet_type_name'])
df_2_3['breed'] = breed.fit_transform(df_2_3['pet_breed_name'])
df_2_3['age'] = age.fit_transform(df_2_3['pet_age_name'])
df_2_3['size'] = size.fit_transform(df_2_3['pet_size_name'])
df_2_3['b'] = b.fit_transform(df_2_3['pet_breed'])
df_2_3['area'] = area.fit_transform(df_2_3['area_name'])
df_2_3['ward'] = ward.fit_transform(df_2_3['ward_name'])
df_2_3['region'] = region.fit_transform(df_2_3['region_name'])
df_2_3

<ipython-input-153-260d790508f3>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_2_3['type'] = type.fit_transform(df_2_3['pet_type_name'])
<ipython-input-153-260d790508f3>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_2_3['breed'] = breed.fit_transform(df_2_3['pet_breed_name'])
<ipython-input-153-260d790508f3>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: 

,pet_breed,pet_breed_name,pet_age_name,pet_size_name,pet_type_name,longitude,latitude,price,area_name,ward_name,region_name,type,breed,age,size,b,area,ward,region
0,17.0,Chó Phốc Sóc,Chó con (dưới 3 tháng tuổi),Siêu nhỏ (tiny),Chó,106.685135,10.950786,5.5,Thành phố Thuận An,Phường An Thạnh,Bình Dương,1,49,2,3,16,284,46,2
1,21.0,Chó Pug,Chó nhỏ (dưới 1 năm tuổi),Nhỏ,Chó,105.69111,21.013826,2.268686,Huyện Hoài Đức,Xã Song Phương,Hà Nội,1,52,3,2,20,58,895,16
2,11.0,Chó Golden,Chó trưởng thành (hơn 1 tuổi),Lớn,Chó,106.59671,10.868425,8.3,Huyện Hóc Môn,Xã Tân Xuân,Tp Hồ Chí Minh,1,40,4,1,10,66,1005,44
3,23.0,Chó Samoyed,Chó nhỏ (dưới 1 năm tuổi),Nhỏ,Chó,107.23118,10.705099,5.5,Huyện Châu Đức,Xã Kim Long,Bà Rịa - Vũng Tàu,1,54,3,2,22,22,732,1
4,3.0,Chó Becgie,Chó nhỏ (dưới 1 năm tuổi),Lớn,Chó,105.94061,21.22481,0.16,Huyện Yên Phong,Xã Tam Giang,Bắc Ninh,1,29,3,1,2,181,907,9
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5303,1.0,Chim Bồ Câu,Chim lớn (từ 3 tháng tuổi),Khác (không xác định được),Chim,106.58481,10.716207,0.22,Huyện Bình Chánh,Xã Tân Kiên,Tp Hồ Chí Minh,0,1,0,0,0,7,980,44
5304,4.0,Chó Bull,Chó con (dưới 3 tháng tuổi),Nhỏ,Chó,106.48673,10.9941,5.0,Huyện Củ Chi,Xã Tân An Hội,Tp Hồ Chí Minh,1,30,2,2,3,42,969,44
5305,8.0,Mèo Sphynx,Mèo con (dưới 3 tháng tuổi),Khác (không xác định được),Mèo,106.62026,10.87035,6.5,Quận 12,Phường Tân Chánh Hiệp,Tp Hồ Chí Minh,3,79,8,0,7,204,388,44
5306,5.0,Chim Họa Mi,Chim lớn (từ 3 tháng tuổi),Khác (không xác định được),Chim,106.77654,10.725062,0.9,Huyện Nhơn Trạch,Xã Phú Hữu,Đồng Nai,0,8,0,0,4,103,843,52


In [ ]:
X = df_2_3.drop(['price', 'pet_type_name', 'pet_age_name', 'pet_size_name', 'pet_breed', 'area_name', 'ward_name', 'region_name', 'pet_breed_name'], axis = 1).values
y = df_2_3.price.values

X[0]

array([106.685135, 10.950786, 1, 49, 2, 3, 16, 284, 46, 2], dtype=object)

In [ ]:
ct = ColumnTransformer([('onehot', OneHotEncoder(), [2, 3, 4, 5, 6, 7, 8, 9])], remainder = 'passthrough')

X = ct.fit_transform(X)
X.toarray()[0]

array([  0.      ,   1.      ,   0.      , ...,   0.      , 106.685135,
        10.950786])

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state=42)

model_3 = LinearRegression()
model_3.fit(X_train, y_train)
c = model_3.score(X_test, y_test)
c

0.41761584809633623

In [ ]:
a, e, c

(0.4168278769152166, 0.2261336868368391, 0.41761584809633623)

# Dựa vào kết quả R^2 -> lựa chọn tập 2.3

# 3. Thực nghiệm trên các mô hình khác nhau

# 3.1 : Các mô hình lựa chọn : Linear, Lasso, Ridge, RandomForest, DecisionTree

In [ ]:
model_params = {
    'linear': {
        'model': LinearRegression(),
        'params' : {
          'fit_intercept': [True, False]
        }
    },
    'L1' : {
        'model': Lasso(),
        'params': {
            'alpha': [0.1, 0.5, 1.0, 5.0, 10.0],
            'max_iter' : [10, 100]
        }
    },
    'L2': {
        'model': Ridge(),
        'params' : {
            'alpha': [0.1, 0.5, 1.0, 5.0, 10.0],
            'max_iter' : [10, 100]
        }
    },
    'RandomForest' : {
        'model' : RandomForestRegressor(),
        'params' : {
            'n_estimators' : [50, 100, 150],
            'min_samples_split' : [2, 5, 10]
        }
    },
    'DecisionTree' : {
        'model' : DecisionTreeRegressor(),
        'params' : {
            'min_samples_split': [2, 5, 10],
            'max_depth': [None, 5, 10],
        }
    }
}

# Áp dụng GridSearchCV để tìm ra bộ tham số tốt nhất cho từng mô hình

In [ ]:
scores = []

for model_name, mp in model_params.items():
    clf =  GridSearchCV(mp['model'], mp['params'], cv=5, return_train_score=False)
    clf.fit(X_train, y_train)
    scores.append({
        'model': model_name,
        'best_score': clf.best_score_,
        'best_params': clf.best_params_
    })

scores

[{'model': 'linear',
  'best_score': 0.362459964172596,
  'best_params': {'fit_intercept': False}},
 {'model': 'L1',
  'best_score': 0.15660915021380895,
  'best_params': {'alpha': 0.1, 'max_iter': 100}},
 {'model': 'L2',
  'best_score': 0.4804160482460317,
  'best_params': {'alpha': 10.0, 'max_iter': 100}},
 {'model': 'RandomForest',
  'best_score': 0.48916241748327227,
  'best_params': {'min_samples_split': 5, 'n_estimators': 150}},
 {'model': 'DecisionTree',
  'best_score': 0.4174129106396302,
  'best_params': {'max_depth': 10, 'min_samples_split': 10}}]

# Áp dụng các tham số tốt nhất đã tìm được để xem kết quả cuối cùng

In [ ]:
linear_reg_model = LinearRegression(fit_intercept=False)
linear_reg_model.fit(X_train, y_train)
linear_reg_predictions = linear_reg_model.predict(X_test)
linear_reg_rmse = mean_squared_error(y_test, linear_reg_predictions, squared=False)
linear_reg_r2 = r2_score(y_test, linear_reg_predictions)
print(f"Linear Regression MSE: {linear_reg_rmse}")
print(f"Linear Regression R-squared: {linear_reg_r2}")
print('=' * 60)

# Lasso Regression (L1)
lasso_reg_model = Lasso(alpha = 0.1, max_iter = 100)
lasso_reg_model.fit(X_train, y_train)
lasso_reg_predictions = lasso_reg_model.predict(X_test)
lasso_reg_rmse = mean_squared_error(y_test, lasso_reg_predictions, squared=False)
lasso_reg_r2 = r2_score(y_test, lasso_reg_predictions)
print(f"Lasso Regression (L1) MSE: {lasso_reg_rmse}")
print(f"Lasso Regression (L1) R-squared: {lasso_reg_r2}")
print('=' * 60)

# Ridge Regression (L2)
ridge_reg_model = Ridge(alpha = 5.0, max_iter = 100)
ridge_reg_model.fit(X_train, y_train)
ridge_reg_predictions = ridge_reg_model.predict(X_test)
ridge_reg_rmse = mean_squared_error(y_test, ridge_reg_predictions, squared=False)
ridge_reg_r2 = r2_score(y_test, ridge_reg_predictions)
print(f"Ridge Regression (L2) MSE: {ridge_reg_rmse}")
print(f"Ridge Regression (L2) R-squared: {ridge_reg_r2}")
print('=' * 60)

# Random Forest
random_forest_model = RandomForestRegressor(random_state=42, min_samples_split = 10, n_estimators = 150)
random_forest_model.fit(X_train, y_train)
random_forest_predictions = random_forest_model.predict(X_test)
random_forest_rmse = mean_squared_error(y_test, random_forest_predictions, squared=False)
random_forest_r2 = r2_score(y_test, random_forest_predictions)
print(f"Random Forest MSE: {random_forest_rmse}")
print(f"Random Forest R-squared: {random_forest_r2}")
print('=' * 60)

# Decision Tree
decision_tree_model = DecisionTreeRegressor(random_state=42, min_samples_split = 10, max_depth = 10)
decision_tree_model.fit(X_train, y_train)
decision_tree_predictions = decision_tree_model.predict(X_test)
decision_tree_rmse = mean_squared_error(y_test, decision_tree_predictions, squared=False)
decision_tree_r2 = r2_score(y_test, decision_tree_predictions)
print(f"Decision Tree MSE: {decision_tree_rmse}")
print(f"Decision Tree R-squared: {decision_tree_r2}")

Linear Regression MSE: 1.5290577646260513
Linear Regression R-squared: 0.4407917800589979
Lasso Regression (L1) MSE: 1.8503224486219454
Lasso Regression (L1) R-squared: 0.18111937187909333
Ridge Regression (L2) MSE: 1.4043770832891125
Ridge Regression (L2) R-squared: 0.528270289705555
Random Forest MSE: 1.4027393376032036
Random Forest R-squared: 0.5293698844373951
Decision Tree MSE: 1.5255649580113846
Decision Tree R-squared: 0.4433436461468484


# Nhận xét :    
# - RandomForest cho ra hiệu suất tốt nhất
# - Linear và DecisionTree cho ra kết quả tương đồng
# - Lasso và Ridge có độ chênh lệch lớn

# Save Model

In [ ]:
import pickle

with open('model_linear','wb') as file:
    pickle.dump(linear_reg_model, file)

with open('model_lasso','wb') as file:
    pickle.dump(lasso_reg_model, file)

with open('model_ridge','wb') as file:
    pickle.dump(ridge_reg_model, file)

with open('model_forest','wb') as file:
    pickle.dump(random_forest_model, file)

with open('model_decision','wb') as file:
    pickle.dump(decision_tree_model, file)

# Save LabelEncoder and OnehotEncoder

In [ ]:
# Save LabelEncoder

with open('pet_breed_name_encoder','wb') as file:
    pickle.dump(breed, file)

with open('pet_age_name_encoder','wb') as file:
    pickle.dump(age, file)

with open('pet_size_name_encoder','wb') as file:
    pickle.dump(size, file)

with open('pet_type_name_encoder','wb') as file:
    pickle.dump(type, file)

with open('pet_breed_encoder','wb') as file:
    pickle.dump(b, file)

with open('area_name_encoder','wb') as file:
    pickle.dump(area, file)

with open('ward_name_encoder','wb') as file:
    pickle.dump(ward, file)

with open('region_name_encoder','wb') as file:
    pickle.dump(region, file)

# Save OneHotEncoder

with open('one_hot_encoder','wb') as file:
    pickle.dump(ct, file)

# Load Model va Encoder

In [3]:
# Load Model

import pickle

with open('./model_linear','rb') as file:
    linear_reg_model = pickle.load(file)

with open('./model_lasso','rb') as file:
    lasso_reg_model = pickle.load(file)

with open('./model_ridge','rb') as file:
    ridge_reg_model = pickle.load(file)

with open('./model_forest','rb') as file:
    random_forest_model = pickle.load(file)

with open('./model_decision','rb') as file:
    decision_tree_model = pickle.load(file)

In [4]:
# Load LabelEncoder

with open('./pet_breed_name_encoder','rb') as file:
    breed_name = pickle.load(file)

with open('./pet_age_name_encoder','rb') as file:
    age_name = pickle.load(file)

with open('./pet_size_name_encoder','rb') as file:
    size_name = pickle.load(file)

with open('./pet_type_name_encoder','rb') as file:
    type_name = pickle.load(file)

with open('./pet_breed_encoder','rb') as file:
    breed = pickle.load(file)

with open('./area_name_encoder','rb') as file:
    area_name = pickle.load(file)

with open('./ward_name_encoder','rb') as file:
    ward_name = pickle.load(file)

with open('./region_name_encoder','rb') as file:
    region_name = pickle.load(file)

# Load OneHotEncoder

with open('./one_hot_encoder','rb') as file:
    ct = pickle.load(file)

In [91]:
ward_name.classes_

array(['Phường 1', 'Phường 10', 'Phường 11', ..., 'Xã Đức Long',
       'Xã Đức Lập Hạ', 'Xã Đức Thượng'], dtype=object)

# Handle_Error

In [5]:
region_name.classes_

array(['An Giang', 'Bà Rịa - Vũng Tàu', 'Bình Dương', 'Bình Phước',
       'Bình Thuận', 'Bình Định', 'Bạc Liêu', 'Bắc Giang', 'Bắc Kạn',
       'Bắc Ninh', 'Bến Tre', 'Cà Mau', 'Cần Thơ', 'Gia Lai', 'Hà Giang',
       'Hà Nam', 'Hà Nội', 'Hà Tĩnh', 'Hưng Yên', 'Hải Dương',
       'Hải Phòng', 'Hậu Giang', 'Khánh Hòa', 'Kiên Giang', 'Kon Tum',
       'Long An', 'Lâm Đồng', 'Lạng Sơn', 'Nam Định', 'Nghệ An',
       'Ninh Bình', 'Ninh Thuận', 'Phú Thọ', 'Phú Yên', 'Quảng Bình',
       'Quảng Nam', 'Quảng Ngãi', 'Quảng Ninh', 'Sóc Trăng', 'Thanh Hóa',
       'Thái Bình', 'Thái Nguyên', 'Thừa Thiên Huế', 'Tiền Giang',
       'Tp Hồ Chí Minh', 'Trà Vinh', 'Tây Ninh', 'Vĩnh Long', 'Vĩnh Phúc',
       'Đà Nẵng', 'Đắk Lắk', 'Đắk Nông', 'Đồng Nai', 'Đồng Tháp'],
      dtype=object)

In [6]:
def handle_pet_breed(value):
  if (value < 1.0) or (value > 33.0) : return False
  return True

def handle_pet_breed_name(value):
  if value not in breed_name.classes_:
    return 'Khác'
  return value

def handle_pet_age_name(value):
  if value not in age_name.classes_:
    return 'Khác (không xác định được)'
  return value

def handle_pet_size_name(value):
  if value not in size_name.classes_:
    return 'Khác (không xác định được)'
  return value

def handle_pet_type_name(value):
  if value not in type_name.classes_:
    return 'Thú cưng khác'
  return value

def handle_area_name(value):
  # Handle Syntax
  l = ['Huyện ', 'Quận ', 'Thị xã ', 'Thành phố ']
  flag_1 = False
  for each in l :
    if each in value : flag_1 = True
  if flag_1 == False :
    print(f'{value} không hợp lệ')

  # Handle Exist Value
  flag_2 = True
  if value not in area_name.classes_ :
    flag_2 = False
    print(f'{value} không tồn tại trong list các thành phố')
  return (flag_1 == True) & (flag_2 == True)

def handle_ward_name(value):
  # Handle Syntax
  l = ['Xã ', 'Thị trấn ', 'Phường ']
  flag_1 = False
  for each in l :
    if each in value : flag_1 = True
  if flag_1 == False :
    print(f'{value} không hợp lệ')

  # Handle Exist Value
  flag_2 = True
  if value not in ward_name.classes_ :
    flag_2 = False
    print(f'{value} không tồn tại trong list các phường xã')
  return (flag_1 == True) & (flag_2 == True)

def handle_region_name(value):
  if value not in region_name.classes_:
    print(f'{value} không tồn tại trong list các tỉnh thành')
    return False
  return True

In [7]:
def check_input(input):
  if handle_pet_breed(input['pet_breed']) == False :
    print('Pet breed phải nằm trong khoảng từ 1 đến 33')
    return False

  if handle_area_name(input['area_name']) == False :
    return False

  if handle_ward_name(input['ward_name']) == False :
    return False

  if handle_region_name(input['region_name']) == False :
    return False

  input['pet_breed_name'] = handle_pet_breed_name(input['pet_breed_name'])
  input['pet_age_name'] = handle_pet_age_name(input['pet_age_name'])
  input['pet_size_name'] = handle_pet_size_name(input['pet_size_name'])
  input['pet_type_name'] = handle_pet_type_name(input['pet_type_name'])
  input

  return True

In [8]:
# Format đúng cho input :
# pet_breed : từ 1.0 đến 33.0
# pet_breed_name : Tên loài + dấu cách + giống (VD : Chó Husky)
# pet_age_name : Tên loài + cách gọi tuổi của từng loài + cách đặt tuổi theo chotot (VD : Chim non dưới 3 tháng tuổi)
# pet_size_name : 'Khác (không xác định được)', 'Lớn', 'Nhỏ', 'Siêu nhỏ (tiny)','Trung bình'
# pet_type_name : Viết hoa chữ cái đầu (VD : Chim, Chó, Tôm, ...)

# area_name : Viết hoa chữ cái đầu + dấu cách + tên (VD : Huyện Mê Linh)
# ward_name : Viết hoa chữ cái đầu + dấu cách + tên (VD : Phường 10)

sample = {
    'pet_breed' : 11.0,
    'pet_breed_name' : 'Tôm hùm alaska',
    'pet_age_name' : '300 tuổi',
    'pet_size_name' : '82 cm',
    'pet_type_name' : 'Tôm',
    'area_name' : 'Quận 4',
    'ward_name' : 'Phường 10',
    'region_name' : 'Tây Ninh',
    'longtitude' : 106.685135,
    'latitude' : 10.950786
}

check_input(sample), sample

(True,
 {'pet_breed': 11.0,
  'pet_breed_name': 'Khác',
  'pet_age_name': 'Khác (không xác định được)',
  'pet_size_name': 'Khác (không xác định được)',
  'pet_type_name': 'Thú cưng khác',
  'area_name': 'Quận 4',
  'ward_name': 'Phường 10',
  'region_name': 'Tây Ninh',
  'longtitude': 106.685135,
  'latitude': 10.950786})

# Model Prediction

In [16]:
def prediction(dic, linear_reg_model, lasso_reg_model, ridge_reg_model, random_forest_model, decision_tree_model):
  pet_breed = breed.transform(np.reshape(dic['pet_breed'], -1))
  pet_breed_name = breed_name.transform(np.reshape(dic['pet_breed_name'], -1))
  pet_age_name = age_name.transform(np.reshape(dic['pet_age_name'], -1))
  pet_size_name = size_name.transform(np.reshape(dic['pet_size_name'], -1))
  pet_type_name = type_name.transform(np.reshape(dic['pet_type_name'], -1))

  longtitude = dic['longtitude']
  latitude = dic['latitude']

  area = area_name.transform(np.reshape(dic['area_name'], -1))
  ward = ward_name.transform(np.reshape(dic['ward_name'], -1))
  region = region_name.transform(np.reshape(dic['region_name'], -1))

  input = np.array([
    longtitude, latitude, pet_type_name[0], pet_breed_name[0], pet_age_name[0], pet_size_name[0], pet_breed[0],
    area[0], ward[0], region[0]
  ]).reshape(1, -1).astype(object)
  input = ct.transform(input)

  output_linear = linear_reg_model.predict(input).item()
  output_lasso = lasso_reg_model.predict(input).item()
  output_ridge = ridge_reg_model.predict(input).item()
  output_forest = random_forest_model.predict(input).item()
  output_decision = decision_tree_model.predict(input).item()

  output = {
      'Linear' : output_linear, 
      'Lasso' : output_lasso, 
      'Ridge' : output_ridge, 
      'Random Forest' : output_forest, 
      'Decision Tree' :  output_decision, 
      'mean': (
          output_linear + 
          output_lasso + 
          output_ridge + 
          output_forest + 
          output_decision
      ) / 6
  }

  return output

In [17]:
prediction(sample, linear_reg_model, lasso_reg_model, ridge_reg_model, random_forest_model, decision_tree_model)

{'Linear': 0.4029496151912506,
 'Lasso': 1.4128881969655198,
 'Ridge': 0.26859181349385297,
 'Random Forest': 0.39101675950003073,
 'Decision Tree': 0.631447314035086,
 'mean': 0.5178156165309568}